# Installation
CliMETAR users some specific packages, that are not available by default. Therefore, you should download and install them before using the other scripts.

This notebook takes care of that. If all packages are installed, no harm will be done.

In [1]:
!conda install -qyS cartopy


# All requested packages already installed.



In [2]:
import climetar

SyntaxError: invalid syntax (quantities.py, line 96)